In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from transformers import AutoTokenizer, AutoModel
import torch
from tqdm import tqdm

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

tokenizer = AutoTokenizer.from_pretrained("allenai/biomed_roberta_base")
model = AutoModel.from_pretrained("allenai/biomed_roberta_base").to(device)

tokenizer_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/430 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/656M [00:00<?, ?B/s]

In [2]:
def replace_longest_text_with_halves(texts):
    if not texts:
        return texts

    max_length_index = max(range(len(texts)), key=lambda i: len(texts[i]))
    longest_text = texts[max_length_index]
    mid_index = len(longest_text) // 2
    first_half = longest_text[:mid_index]
    second_half = longest_text[mid_index:]
    texts[max_length_index:max_length_index + 1] = [first_half, second_half]
    return texts

def process_text_vectorization_with_pad(text_list):
    cls_list = []
    for i in tqdm(range(len(text_list))):
        sentences = text_list[i].split('. ')
        tokenized = tokenizer(sentences, add_special_tokens=True, padding = True, return_tensors="pt")
        tokenized = {k:torch.tensor(v).to(device) for k,v in tokenized.items()}

        if tokenized['input_ids'].size(dim=1) > 512:
            sentences_new = replace_longest_text_with_halves(sentences)
            tokenized_new = tokenizer(sentences_new, add_special_tokens=True, padding = True, return_tensors="pt")
            tokenized_new = {k:torch.tensor(v).to(device) for k,v in tokenized_new.items()}
            with torch.no_grad():
                hidden_state_new = model(**tokenized_new)
            cls_state_new = hidden_state_new.last_hidden_state[:,0,:]
            cls_state_new = cls_state_new.to('cpu')
            cls_list.append(torch.unsqueeze(torch.mean(cls_state_new, dim = 0), 0))

        else:
            with torch.no_grad():
                hidden_state = model(**tokenized)
            cls_state = hidden_state.last_hidden_state[:,0,:]
            cls_state = cls_state.to('cpu')
            cls_list.append(torch.unsqueeze(torch.mean(cls_state, dim = 0), 0))
    return cls_list

In [3]:
title_abstract_texts_X = pd.read_csv('title_abstract_texts_X.csv', sep=',')
title_abstract_texts_all = pd.read_csv('title_abstract_texts_all.csv', sep=',')

In [4]:
texts_X = list(title_abstract_texts_X['0'].values)
all_texts = list(title_abstract_texts_all['0'].values)

In [5]:
def add_space_after_period(text):
    return text.replace('.', '. ')

for i in tqdm(range(len(texts_X))):
    abst_sent = texts_X[i].split('. ')
    sent_len = []
    for k in range(len(abst_sent)):
        sent_len.append(len(abst_sent[k]))
    if max(sent_len) > 512:
        texts_X[i] = add_space_after_period(texts_X[i])

100%|██████████| 8676/8676 [00:00<00:00, 201278.71it/s]


In [6]:
text_vect_X = process_text_vectorization_with_pad(texts_X)

100%|██████████| 8676/8676 [03:58<00:00, 36.42it/s]


In [7]:
cls_X_array = []
for i in range(len(text_vect_X)):
    cls_X_array.append(text_vect_X[i][0].numpy())
cls_X_array = np.array(cls_X_array)
cls_X_matrix = pd.DataFrame(cls_X_array)
cls_X_matrix

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,0.004396,0.111531,-0.053759,-0.122934,0.094525,0.029950,-0.026960,-0.049314,0.080843,-0.138057,...,-0.001560,0.001530,-0.183092,-0.046512,-0.003630,0.055863,0.119410,-0.110952,-0.047611,-0.062239
1,-0.015581,0.146944,-0.047382,-0.098750,0.091179,0.018148,-0.003952,-0.017282,0.047970,-0.138498,...,-0.011372,-0.027085,-0.157765,-0.025209,-0.022628,0.049641,0.093265,-0.176488,-0.034717,-0.081885
2,-0.049539,0.136395,-0.063148,-0.136461,0.087696,-0.005887,-0.033455,-0.030302,0.018284,-0.162250,...,0.023941,0.008050,-0.191988,-0.063020,0.003906,0.019137,0.084069,-0.141838,-0.037537,-0.095892
3,-0.047433,0.159374,-0.040301,-0.104408,0.119928,0.007361,-0.014051,-0.015842,0.046419,-0.134662,...,-0.028223,0.004081,-0.160321,-0.044982,-0.033107,0.044825,0.074023,-0.168744,-0.061473,-0.084036
4,-0.002795,0.138985,-0.069742,-0.102418,0.099182,0.000717,-0.019203,-0.038088,0.026243,-0.145042,...,0.007437,-0.012597,-0.169961,-0.032344,-0.001023,0.045827,0.090556,-0.168094,-0.015017,-0.069998
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8671,-0.045336,0.133677,-0.029134,-0.107541,0.132256,0.051750,-0.018637,-0.040218,0.059308,-0.170887,...,0.022084,0.032664,-0.233980,-0.059937,-0.004573,0.017550,0.097466,-0.174400,-0.021677,-0.063707
8672,-0.045177,0.152992,-0.041098,-0.104705,0.127691,0.024053,-0.030701,-0.064440,0.088367,-0.164559,...,-0.007433,0.041335,-0.193963,-0.045986,-0.003841,0.031248,0.060407,-0.175191,-0.028460,-0.071196
8673,-0.049712,0.103376,-0.031867,-0.095337,0.121834,0.005025,-0.009786,-0.019835,0.059511,-0.172024,...,0.008343,0.023531,-0.203889,-0.020670,-0.012049,0.043372,0.065292,-0.214333,-0.057071,-0.064413
8674,-0.054738,0.124515,-0.013116,-0.114506,0.119348,0.005930,-0.014924,-0.031042,0.046502,-0.156178,...,0.009959,0.030476,-0.201300,-0.050125,-0.014751,0.015561,0.098714,-0.217591,-0.030193,-0.085523


In [8]:
cls_X_matrix.to_csv('cls_X_matrix_biomed_roberta.csv', index=False)

In [9]:
for i in tqdm(range(len(all_texts))):
    abst_sent = all_texts[i].split('. ')
    sent_len = []
    for k in range(len(abst_sent)):
        sent_len.append(len(abst_sent[k]))
    if max(sent_len) > 512:
        all_texts[i] = add_space_after_period(all_texts[i])

100%|██████████| 73154/73154 [00:00<00:00, 120285.96it/s]


In [10]:
vect_abs_all = process_text_vectorization_with_pad(all_texts)

100%|██████████| 73154/73154 [37:39<00:00, 32.38it/s]


In [11]:
cls_all_array = []
for i in range(len(vect_abs_all)):
    cls_all_array.append(vect_abs_all[i][0].numpy())
cls_all_array = np.array(cls_all_array)
cls_all_matrix = pd.DataFrame(cls_all_array)
cls_all_matrix

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,-0.045195,0.126076,-0.053730,-0.105506,0.159888,0.048048,-0.006418,-0.048368,0.042918,-0.167247,...,0.021563,-0.034194,-0.217675,-0.072104,-0.011408,0.072525,0.084023,-0.198236,-0.006293,-0.084537
1,-0.023131,0.129438,-0.035607,-0.086480,0.148970,0.073451,0.000565,-0.008266,-0.001043,-0.174358,...,-0.048355,-0.023552,-0.182926,-0.064953,-0.003158,0.042354,0.098815,-0.193225,0.002935,-0.094729
2,-0.032069,0.114485,-0.065266,-0.130257,0.084599,0.023531,-0.029004,-0.055035,0.052943,-0.154862,...,0.012632,-0.033500,-0.185202,-0.024105,0.003597,0.052702,0.052254,-0.177673,-0.028796,-0.069861
3,-0.013892,0.106585,-0.051791,-0.096179,0.112613,0.065832,-0.016044,-0.060396,0.056556,-0.149652,...,-0.006343,-0.014142,-0.182443,-0.037349,0.017616,0.046790,0.048526,-0.190286,-0.008112,-0.078073
4,-0.029579,0.124763,-0.062911,-0.122002,0.134878,0.038389,0.004006,-0.068102,0.037825,-0.170457,...,-0.010585,0.004920,-0.204565,-0.077281,-0.028932,0.057288,0.045457,-0.185456,-0.004856,-0.086491
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73149,-0.034707,0.121593,-0.048980,-0.089716,0.122813,0.035930,-0.016205,-0.004211,0.085050,-0.158890,...,0.007231,0.019344,-0.146664,0.005215,0.034134,0.017782,0.056271,-0.177068,-0.053055,-0.047713
73150,-0.063084,0.151562,-0.023835,-0.116318,0.133180,0.015429,-0.032492,-0.045453,0.074657,-0.154053,...,-0.008152,0.046478,-0.174762,-0.043759,-0.021120,0.008394,0.090016,-0.177932,-0.041945,-0.061915
73151,-0.060750,0.141699,-0.024987,-0.123299,0.126260,0.037649,-0.030914,-0.047911,0.082710,-0.161198,...,0.012582,0.039903,-0.163820,-0.049184,0.025995,0.035591,0.091056,-0.178178,-0.037770,-0.054753
73152,-0.000120,0.135447,-0.045649,-0.089250,0.146096,0.033064,-0.021247,-0.013789,0.072819,-0.168562,...,-0.008896,-0.001153,-0.181171,-0.047848,0.002495,0.010382,0.071604,-0.167993,-0.029465,-0.081822


In [12]:
cls_all_matrix.to_csv('cls_all_matrix_biomed_roberta.csv', index=False)